# SQLGlot Explainer: Snowflake to BigQuery Transpilation

This notebook demonstrates how to use SQLGlot to transpile SQL queries from Snowflake syntax to BigQuery syntax.

## Overview

SQLGlot is a no-dependency SQL parser, transpiler, and optimizer. It can parse SQL queries from various dialects, convert them to different SQL dialects, and optimize them.

### Key Features:
- Parse SQL from one dialect (e.g., Snowflake)
- Transpile to another dialect (e.g., BigQuery)
- Handle syntax differences automatically
- Preserve query semantics

## What You'll Learn

1. Basic transpilation between Snowflake and BigQuery
2. Common syntax differences between the two dialects
3. Handling complex queries (JOINs, window functions, etc.)
4. Identifying edge cases and manual adjustments needed

---

**Note**: This notebook does NOT connect to any databases. We're only working with SQL strings.

In [1]:
# Install/import required libraries
import sqlglot
from sqlglot import exp, parse_one, transpile
from sqlglot.dialects import Snowflake, BigQuery
from pprint import pprint

print(f"SQLGlot version: {sqlglot.__version__}")

SQLGlot version: 28.6.0


## 1. Basic Transpilation

Let's start with a simple SELECT query and see how SQLGlot converts it from Snowflake to BigQuery.

In [2]:
# Example 1: Simple SELECT query
snowflake_sql = """
SELECT 
    id,
    name,
    email
FROM users
WHERE active = TRUE
LIMIT 100
"""

print("=" * 80)
print("SNOWFLAKE SQL:")
print("=" * 80)
print(snowflake_sql)

# Transpile from Snowflake to BigQuery
bigquery_sql = transpile(
    snowflake_sql,
    read=Snowflake,
    write=BigQuery,
    pretty=True
)[0]

print("\n" + "=" * 80)
print("BIGQUERY SQL:")
print("=" * 80)
print(bigquery_sql)

SNOWFLAKE SQL:

SELECT 
    id,
    name,
    email
FROM users
WHERE active = TRUE
LIMIT 100


BIGQUERY SQL:
SELECT
  id,
  name,
  email
FROM users
WHERE
  active = TRUE
LIMIT 100


## 2. Data Type Differences

Snowflake and BigQuery have different data type names. SQLGlot handles most conversions automatically.

In [3]:
# Example 2: CREATE TABLE with data types
snowflake_create_table = """
CREATE TABLE products (
    id NUMBER(38, 0),
    name VARCHAR(255),
    price DECIMAL(10, 2),
    description TEXT,
    created_at TIMESTAMP_NTZ,
    metadata VARIANT
)
"""

print("=" * 80)
print("SNOWFLAKE CREATE TABLE:")
print("=" * 80)
print(snowflake_create_table)

bigquery_create_table = transpile(
    snowflake_create_table,
    read=Snowflake,
    write=BigQuery,
    pretty=True
)[0]

print("\n" + "=" * 80)
print("BIGQUERY CREATE TABLE:")
print("=" * 80)
print(bigquery_create_table)

print("\n" + "=" * 80)
print("KEY DIFFERENCES NOTED:")
print("=" * 80)
print("• NUMBER → INT64 or NUMERIC (depending on precision)")
print("• VARCHAR → STRING")
print("• TEXT → STRING")
print("• TIMESTAMP_NTZ → TIMESTAMP")
print("• VARIANT → JSON (BigQuery uses JSON type)")

SNOWFLAKE CREATE TABLE:

CREATE TABLE products (
    id NUMBER(38, 0),
    name VARCHAR(255),
    price DECIMAL(10, 2),
    description TEXT,
    created_at TIMESTAMP_NTZ,
    metadata VARIANT
)


BIGQUERY CREATE TABLE:
CREATE TABLE products (
  id NUMERIC(38, 0),
  name STRING(255),
  price NUMERIC(10, 2),
  description STRING,
  created_at DATETIME,
  metadata ANY TYPE
)

KEY DIFFERENCES NOTED:
• NUMBER → INT64 or NUMERIC (depending on precision)
• VARCHAR → STRING
• TEXT → STRING
• TIMESTAMP_NTZ → TIMESTAMP
• VARIANT → JSON (BigQuery uses JSON type)


## 3. String Functions and Concatenation

Snowflake uses `||` for string concatenation (like Oracle), while BigQuery uses `CONCAT()` function. SQLGlot handles this conversion.

In [4]:
# Example 3: String concatenation
snowflake_concat = """
SELECT 
    first_name || ' ' || last_name AS full_name,
    email || '@company.com' AS company_email
FROM users
"""

print("=" * 80)
print("SNOWFLAKE SQL (using || operator):")
print("=" * 80)
print(snowflake_concat)

bigquery_concat = transpile(
    snowflake_concat,
    read=Snowflake,
    write=BigQuery,
    pretty=True
)[0]

print("\n" + "=" * 80)
print("BIGQUERY SQL (using CONCAT function):")
print("=" * 80)
print(bigquery_concat)

SNOWFLAKE SQL (using || operator):

SELECT 
    first_name || ' ' || last_name AS full_name,
    email || '@company.com' AS company_email
FROM users


BIGQUERY SQL (using CONCAT function):
SELECT
  first_name || ' ' || last_name AS full_name,
  email || '@company.com' AS company_email
FROM users


## 4. Date and Time Functions

Date functions differ between Snowflake and BigQuery. Let's examine some common date operations.

In [5]:
# Example 4: Date functions
snowflake_date = """
SELECT 
    CURRENT_DATE() AS today,
    DATEADD(day, -7, CURRENT_DATE()) AS last_week,
    DATEDIFF(day, created_at, CURRENT_DATE()) AS days_ago,
    DATE_TRUNC('month', created_at) AS month_start,
    EXTRACT(YEAR FROM created_at) AS year
FROM orders
"""

print("=" * 80)
print("SNOWFLAKE SQL:")
print("=" * 80)
print(snowflake_date)

bigquery_date = transpile(
    snowflake_date,
    read=Snowflake,
    write=BigQuery,
    pretty=True
)[0]

print("\n" + "=" * 80)
print("BIGQUERY SQL:")
print("=" * 80)
print(bigquery_date)

print("\n" + "=" * 80)
print("FUNCTION MAPPINGS:")
print("=" * 80)
print("• DATEADD(day, -7, date) → DATE_SUB(date, INTERVAL 7 DAY)")
print("• DATEDIFF(day, date1, date2) → DATE_DIFF(date2, date1, DAY)")
print("• DATE_TRUNC('month', date) → DATE_TRUNC(date, MONTH)")
print("• EXTRACT(YEAR FROM date) → EXTRACT(YEAR FROM date) (same)")

SNOWFLAKE SQL:

SELECT 
    CURRENT_DATE() AS today,
    DATEADD(day, -7, CURRENT_DATE()) AS last_week,
    DATEDIFF(day, created_at, CURRENT_DATE()) AS days_ago,
    DATE_TRUNC('month', created_at) AS month_start,
    EXTRACT(YEAR FROM created_at) AS year
FROM orders


BIGQUERY SQL:
SELECT
  CURRENT_DATE AS today,
  DATE_ADD(CURRENT_DATE, INTERVAL -7 DAY) AS last_week,
  DATE_DIFF(CURRENT_DATE, created_at, DAY) AS days_ago,
  TIMESTAMP_TRUNC(created_at, MONTH) AS month_start,
  EXTRACT(YEAR FROM created_at) AS year
FROM orders

FUNCTION MAPPINGS:
• DATEADD(day, -7, date) → DATE_SUB(date, INTERVAL 7 DAY)
• DATEDIFF(day, date1, date2) → DATE_DIFF(date2, date1, DAY)
• DATE_TRUNC('month', date) → DATE_TRUNC(date, MONTH)
• EXTRACT(YEAR FROM date) → EXTRACT(YEAR FROM date) (same)


## 5. Window Functions

Window functions work similarly in both dialects, but there might be subtle differences in syntax.

In [6]:
# Example 5: Window functions
snowflake_window = """
SELECT 
    id,
    user_id,
    amount,
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY created_at DESC) AS rn,
    SUM(amount) OVER (PARTITION BY user_id) AS user_total,
    AVG(amount) OVER (ORDER BY created_at ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS rolling_avg
FROM orders
"""

print("=" * 80)
print("SNOWFLAKE SQL:")
print("=" * 80)
print(snowflake_window)

bigquery_window = transpile(
    snowflake_window,
    read=Snowflake,
    write=BigQuery,
    pretty=True
)[0]

print("\n" + "=" * 80)
print("BIGQUERY SQL:")
print("=" * 80)
print(bigquery_window)

print("\n" + "=" * 80)
print("NOTE:")
print("=" * 80)
print("Window functions are generally compatible between Snowflake and BigQuery.")
print("SQLGlot preserves the window function syntax where it's the same.")

'NULLS LAST' translation not supported in window functions


SNOWFLAKE SQL:

SELECT 
    id,
    user_id,
    amount,
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY created_at DESC) AS rn,
    SUM(amount) OVER (PARTITION BY user_id) AS user_total,
    AVG(amount) OVER (ORDER BY created_at ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS rolling_avg
FROM orders


BIGQUERY SQL:
SELECT
  id,
  user_id,
  amount,
  ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY created_at DESC NULLS FIRST) AS rn,
  SUM(amount) OVER (PARTITION BY user_id) AS user_total,
  AVG(amount) OVER (ORDER BY created_at ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS rolling_avg
FROM orders

NOTE:
Window functions are generally compatible between Snowflake and BigQuery.
SQLGlot preserves the window function syntax where it's the same.


## 6. JOINs and Table Aliases

JOIN syntax is largely compatible, but table qualifiers and identifiers might need adjustment.

In [7]:
# Example 6: JOINs with table aliases
snowflake_join = """
SELECT 
    o.order_id,
    o.amount,
    u.email,
    p.product_name
FROM orders o
INNER JOIN users u ON o.user_id = u.id
LEFT JOIN order_items oi ON o.order_id = oi.order_id
LEFT JOIN products p ON oi.product_id = p.id
WHERE o.created_at >= '2024-01-01'
"""

print("=" * 80)
print("SNOWFLAKE SQL:")
print("=" * 80)
print(snowflake_join)

bigquery_join = transpile(
    snowflake_join,
    read=Snowflake,
    write=BigQuery,
    pretty=True
)[0]

print("\n" + "=" * 80)
print("BIGQUERY SQL:")
print("=" * 80)
print(bigquery_join)

SNOWFLAKE SQL:

SELECT 
    o.order_id,
    o.amount,
    u.email,
    p.product_name
FROM orders o
INNER JOIN users u ON o.user_id = u.id
LEFT JOIN order_items oi ON o.order_id = oi.order_id
LEFT JOIN products p ON oi.product_id = p.id
WHERE o.created_at >= '2024-01-01'


BIGQUERY SQL:
SELECT
  o.order_id,
  o.amount,
  u.email,
  p.product_name
FROM orders AS o
INNER JOIN users AS u
  ON o.user_id = u.id
LEFT JOIN order_items AS oi
  ON o.order_id = oi.order_id
LEFT JOIN products AS p
  ON oi.product_id = p.id
WHERE
  o.created_at >= '2024-01-01'


## 7. Array and JSON Functions

Snowflake and BigQuery handle arrays and JSON differently. SQLGlot attempts to map these appropriately.

In [8]:
# Example 7: Array and JSON operations
snowflake_array_json = """
SELECT 
    id,
    ARRAY_SIZE(tags) AS tag_count,
    tags[0] AS first_tag,
    OBJECT_KEYS(metadata) AS metadata_keys,
    GET(metadata, 'status') AS status
FROM products
WHERE ARRAY_CONTAINS(tags, 'featured') = TRUE
"""

print("=" * 80)
print("SNOWFLAKE SQL:")
print("=" * 80)
print(snowflake_array_json)

try:
    bigquery_array_json = transpile(
        snowflake_array_json,
        read=Snowflake,
        write=BigQuery,
        pretty=True
    )[0]
    
    print("\n" + "=" * 80)
    print("BIGQUERY SQL:")
    print("=" * 80)
    print(bigquery_array_json)
except Exception as e:
    print(f"\n⚠️  TRANSPILATION ISSUE:")
    print(f"   {e}")
    print("\n" + "=" * 80)
    print("MANUAL CONVERSION NEEDED:")
    print("=" * 80)
    print("• ARRAY_SIZE() → ARRAY_LENGTH() in BigQuery")
    print("• tags[0] → tags[OFFSET(0)] in BigQuery")
    print("• OBJECT_KEYS() → JSON_EXTRACT_SCALAR() + different approach")
    print("• GET() → JSON_EXTRACT_SCALAR() in BigQuery")
    print("• ARRAY_CONTAINS() → 'value' IN UNNEST(tags) in BigQuery")

SNOWFLAKE SQL:

SELECT 
    id,
    ARRAY_SIZE(tags) AS tag_count,
    tags[0] AS first_tag,
    OBJECT_KEYS(metadata) AS metadata_keys,
    GET(metadata, 'status') AS status
FROM products
WHERE ARRAY_CONTAINS(tags, 'featured') = TRUE


BIGQUERY SQL:
SELECT
  id,
  ARRAY_LENGTH(tags) AS tag_count,
  tags[0] AS first_tag,
  JSON_KEYS(metadata) AS metadata_keys,
  JSON_EXTRACT(metadata, '$.status') AS status
FROM products
WHERE
  EXISTS(
    SELECT
      1
    FROM UNNEST('featured') AS _col
    WHERE
      _col = tags
  ) = TRUE


## 8. CTEs (Common Table Expressions)

CTEs work similarly in both dialects, but let's verify the transpilation.

In [9]:
# Example 8: CTEs (Common Table Expressions)
snowflake_cte = """
WITH active_users AS (
    SELECT id, email, created_at
    FROM users
    WHERE active = TRUE
),
recent_orders AS (
    SELECT user_id, SUM(amount) AS total_amount
    FROM orders
    WHERE created_at >= DATEADD(day, -30, CURRENT_DATE())
    GROUP BY user_id
)
SELECT 
    u.id,
    u.email,
    COALESCE(o.total_amount, 0) AS order_total
FROM active_users u
LEFT JOIN recent_orders o ON u.id = o.user_id
ORDER BY order_total DESC
"""

print("=" * 80)
print("SNOWFLAKE SQL:")
print("=" * 80)
print(snowflake_cte)

bigquery_cte = transpile(
    snowflake_cte,
    read=Snowflake,
    write=BigQuery,
    pretty=True
)[0]

print("\n" + "=" * 80)
print("BIGQUERY SQL:")
print("=" * 80)
print(bigquery_cte)

SNOWFLAKE SQL:

WITH active_users AS (
    SELECT id, email, created_at
    FROM users
    WHERE active = TRUE
),
recent_orders AS (
    SELECT user_id, SUM(amount) AS total_amount
    FROM orders
    WHERE created_at >= DATEADD(day, -30, CURRENT_DATE())
    GROUP BY user_id
)
SELECT 
    u.id,
    u.email,
    COALESCE(o.total_amount, 0) AS order_total
FROM active_users u
LEFT JOIN recent_orders o ON u.id = o.user_id
ORDER BY order_total DESC


BIGQUERY SQL:
WITH active_users AS (
  SELECT
    id,
    email,
    created_at
  FROM users
  WHERE
    active = TRUE
), recent_orders AS (
  SELECT
    user_id,
    SUM(amount) AS total_amount
  FROM orders
  WHERE
    created_at >= DATE_ADD(CURRENT_DATE, INTERVAL -30 DAY)
  GROUP BY
    user_id
)
SELECT
  u.id,
  u.email,
  COALESCE(o.total_amount, 0) AS order_total
FROM active_users AS u
LEFT JOIN recent_orders AS o
  ON u.id = o.user_id
ORDER BY
  order_total DESC NULLS FIRST


## 9. Conditional Logic: CASE and IF Functions

Both dialects support CASE statements, but function-based conditionals differ.

In [10]:
# Example 9: Conditional logic
snowflake_conditional = """
SELECT 
    id,
    CASE 
        WHEN amount > 1000 THEN 'high'
        WHEN amount > 100 THEN 'medium'
        ELSE 'low'
    END AS order_tier,
    IFF(status = 'completed', amount, 0) AS completed_amount,
    NULLIF(description, '') AS description_or_null
FROM orders
"""

print("=" * 80)
print("SNOWFLAKE SQL:")
print("=" * 80)
print(snowflake_conditional)

bigquery_conditional = transpile(
    snowflake_conditional,
    read=Snowflake,
    write=BigQuery,
    pretty=True
)[0]

print("\n" + "=" * 80)
print("BIGQUERY SQL:")
print("=" * 80)
print(bigquery_conditional)

print("\n" + "=" * 80)
print("FUNCTION MAPPINGS:")
print("=" * 80)
print("• IFF(condition, true_val, false_val) → IF(condition, true_val, false_val)")
print("• NULLIF(value1, value2) → NULLIF(value1, value2) (same)")

SNOWFLAKE SQL:

SELECT 
    id,
    CASE 
        WHEN amount > 1000 THEN 'high'
        WHEN amount > 100 THEN 'medium'
        ELSE 'low'
    END AS order_tier,
    IFF(status = 'completed', amount, 0) AS completed_amount,
    NULLIF(description, '') AS description_or_null
FROM orders


BIGQUERY SQL:
SELECT
  id,
  CASE WHEN amount > 1000 THEN 'high' WHEN amount > 100 THEN 'medium' ELSE 'low' END AS order_tier,
  IF(status = 'completed', amount, 0) AS completed_amount,
  NULLIF(description, '') AS description_or_null
FROM orders

FUNCTION MAPPINGS:
• IFF(condition, true_val, false_val) → IF(condition, true_val, false_val)
• NULLIF(value1, value2) → NULLIF(value1, value2) (same)


## 10. Handling Identifier Quoting

Snowflake and BigQuery handle identifier quoting differently. Snowflake uses double quotes by default, while BigQuery uses backticks.

In [11]:
# Example 10: Identifier quoting
snowflake_quoted = """
SELECT 
    "User ID" AS user_id,
    "First Name" AS first_name,
    "Email Address" AS email
FROM "Public"."Users"
WHERE "Active Status" = TRUE
"""

print("=" * 80)
print("SNOWFLAKE SQL (using double quotes):")
print("=" * 80)
print(snowflake_quoted)

bigquery_quoted = transpile(
    snowflake_quoted,
    read=Snowflake,
    write=BigQuery,
    pretty=True
)[0]

print("\n" + "=" * 80)
print("BIGQUERY SQL (using backticks):")
print("=" * 80)
print(bigquery_quoted)

print("\n" + "=" * 80)
print("IDENTIFIER QUOTING:")
print("=" * 80)
print("• Snowflake: Uses double quotes (\"identifier\")")
print("• BigQuery: Uses backticks (`identifier`)")
print("• SQLGlot automatically converts between them")

SNOWFLAKE SQL (using double quotes):

SELECT 
    "User ID" AS user_id,
    "First Name" AS first_name,
    "Email Address" AS email
FROM "Public"."Users"
WHERE "Active Status" = TRUE


BIGQUERY SQL (using backticks):
SELECT
  `User ID` AS user_id,
  `First Name` AS first_name,
  `Email Address` AS email
FROM `Public`.`Users`
WHERE
  `Active Status` = TRUE

IDENTIFIER QUOTING:
• Snowflake: Uses double quotes ("identifier")
• BigQuery: Uses backticks (`identifier`)
• SQLGlot automatically converts between them


## 11. Complex Query: Combining Multiple Features

Let's create a more complex query that combines multiple features to see how SQLGlot handles the full transpilation.

In [12]:
# Example 11: Complex query combining multiple features
snowflake_complex = """
WITH monthly_sales AS (
    SELECT 
        DATE_TRUNC('month', created_at) AS month,
        user_id,
        SUM(amount) AS total_amount,
        COUNT(*) AS order_count
    FROM orders
    WHERE created_at >= DATEADD(month, -12, CURRENT_DATE())
    GROUP BY 1, 2
),
user_rankings AS (
    SELECT 
        month,
        user_id,
        total_amount,
        ROW_NUMBER() OVER (PARTITION BY month ORDER BY total_amount DESC) AS user_rank,
        SUM(total_amount) OVER (PARTITION BY month) AS monthly_total
    FROM monthly_sales
)
SELECT 
    u.email,
    ur.month,
    ur.total_amount,
    ur.user_rank,
    ROUND(ur.total_amount / ur.monthly_total * 100, 2) || '%' AS pct_of_monthly
FROM user_rankings ur
INNER JOIN users u ON ur.user_id = u.id
WHERE ur.user_rank <= 10
ORDER BY ur.month DESC, ur.user_rank ASC
"""

print("=" * 80)
print("SNOWFLAKE SQL (Complex Query):")
print("=" * 80)
print(snowflake_complex)

bigquery_complex = transpile(
    snowflake_complex,
    read=Snowflake,
    write=BigQuery,
    pretty=True
)[0]

print("\n" + "=" * 80)
print("BIGQUERY SQL (Transpiled):")
print("=" * 80)
print(bigquery_complex)

SNOWFLAKE SQL (Complex Query):

WITH monthly_sales AS (
    SELECT 
        DATE_TRUNC('month', created_at) AS month,
        user_id,
        SUM(amount) AS total_amount,
        COUNT(*) AS order_count
    FROM orders
    WHERE created_at >= DATEADD(month, -12, CURRENT_DATE())
    GROUP BY 1, 2
),
user_rankings AS (
    SELECT 
        month,
        user_id,
        total_amount,
        ROW_NUMBER() OVER (PARTITION BY month ORDER BY total_amount DESC) AS user_rank,
        SUM(total_amount) OVER (PARTITION BY month) AS monthly_total
    FROM monthly_sales
)
SELECT 
    u.email,
    ur.month,
    ur.total_amount,
    ur.user_rank,
    ROUND(ur.total_amount / ur.monthly_total * 100, 2) || '%' AS pct_of_monthly
FROM user_rankings ur
INNER JOIN users u ON ur.user_id = u.id
WHERE ur.user_rank <= 10
ORDER BY ur.month DESC, ur.user_rank ASC


BIGQUERY SQL (Transpiled):
WITH monthly_sales AS (
  SELECT
    TIMESTAMP_TRUNC(created_at, MONTH) AS month,
    user_id,
    SUM(amount) AS total_a

## 12. Parsing and AST Inspection

SQLGlot can parse SQL into an Abstract Syntax Tree (AST), which allows for programmatic manipulation.

In [13]:
# Example 12: Parsing and AST inspection
sql = "SELECT id, name FROM users WHERE active = TRUE LIMIT 100"

# Parse into AST
ast = parse_one(sql, read=Snowflake)

print("=" * 80)
print("ORIGINAL SQL:")
print("=" * 80)
print(sql)

print("\n" + "=" * 80)
print("AST STRUCTURE:")
print("=" * 80)
print(f"Root type: {type(ast).__name__}")
print(f"Number of expressions: {len(ast.expressions)}")

print("\n" + "=" * 80)
print("TABLE REFERENCES:")
print("=" * 80)
tables = ast.find_all(exp.Table)
for table in tables:
    print(f"  • {table}")

print("\n" + "=" * 80)
print("COLUMN REFERENCES:")
print("=" * 80)
columns = ast.find_all(exp.Column)
for col in columns:
    print(f"  • {col}")

# Modify the AST (e.g., add a condition)
where_clause = ast.find(exp.Where)
if where_clause:
    print("\n" + "=" * 80)
    print("WHERE CLAUSE FOUND:")
    print("=" * 80)
    print(f"  {where_clause}")

# Convert back to SQL
print("\n" + "=" * 80)
print("RECONSTRUCTED SQL:")
print("=" * 80)
print(ast.sql(dialect=Snowflake))

ORIGINAL SQL:
SELECT id, name FROM users WHERE active = TRUE LIMIT 100

AST STRUCTURE:
Root type: Select
Number of expressions: 2

TABLE REFERENCES:
  • users

COLUMN REFERENCES:
  • id
  • name
  • active

WHERE CLAUSE FOUND:
  WHERE active = TRUE

RECONSTRUCTED SQL:
SELECT id, name FROM users WHERE active = TRUE LIMIT 100


## 13. Error Handling and Edge Cases

Some SQL constructs might not translate perfectly. Let's examine how SQLGlot handles edge cases.

In [14]:
# Example 13: Handling potential issues
test_cases = [
    # Test case 1: PIVOT operation (handled differently)
    """
    SELECT * FROM (
        SELECT category, amount FROM sales
    ) PIVOT (
        SUM(amount) FOR category IN ('A', 'B', 'C')
    ) AS p
    """,
    
    # Test case 2: QUALIFY clause (Snowflake-specific)
    """
    SELECT id, amount, ROW_NUMBER() OVER (ORDER BY amount DESC) AS rn
    FROM orders
    QUALIFY rn <= 10
    """,
]

for i, test_sql in enumerate(test_cases, 1):
    print("=" * 80)
    print(f"TEST CASE {i}:")
    print("=" * 80)
    print(test_sql.strip())
    
    try:
        result = transpile(
            test_sql,
            read=Snowflake,
            write=BigQuery,
            pretty=True
        )[0]
        
        print("\n✅ TRANSPILATION SUCCESS:")
        print(result)
    except Exception as e:
        print(f"\n⚠️  TRANSPILATION FAILED:")
        print(f"   Error: {e}")
        print("\n💡 MANUAL CONVERSION REQUIRED")
    
    print("\n")

TEST CASE 1:
SELECT * FROM (
        SELECT category, amount FROM sales
    ) PIVOT (
        SUM(amount) FOR category IN ('A', 'B', 'C')
    ) AS p

✅ TRANSPILATION SUCCESS:
SELECT
  *
FROM (
  SELECT
    category,
    amount
  FROM sales
)
PIVOT(SUM(amount) FOR category IN ('A', 'B', 'C')) AS p


TEST CASE 2:
SELECT id, amount, ROW_NUMBER() OVER (ORDER BY amount DESC) AS rn
    FROM orders
    QUALIFY rn <= 10

✅ TRANSPILATION SUCCESS:
SELECT
  id,
  amount,
  ROW_NUMBER() OVER (ORDER BY amount DESC NULLS FIRST) AS rn
FROM orders
QUALIFY
  rn <= 10




## Summary: Key Differences Between Snowflake and BigQuery

### ✅ Automatically Handled by SQLGlot

1. **String Concatenation**: `||` → `CONCAT()`
2. **Date Functions**: `DATEADD()`, `DATEDIFF()` → `DATE_ADD()`, `DATE_DIFF()`
3. **Conditional Functions**: `IFF()` → `IF()`
4. **Identifier Quoting**: Double quotes → Backticks
5. **Data Types**: `NUMBER`, `VARCHAR`, `TEXT` → `INT64`, `NUMERIC`, `STRING`
6. **TIMESTAMP_NTZ**: → `TIMESTAMP`

### ⚠️  May Require Manual Adjustment

1. **Array Functions**: `ARRAY_SIZE()`, `ARRAY_CONTAINS()` → Different syntax in BigQuery
2. **JSON Functions**: `OBJECT_KEYS()`, `GET()` → Use `JSON_EXTRACT_SCALAR()` in BigQuery
3. **PIVOT Operations**: Different syntax between dialects
4. **QUALIFY Clause**: Snowflake-specific, needs rewriting in BigQuery
5. **VARIANT Type**: May need explicit JSON handling in BigQuery

### 💡 Best Practices

1. **Always Review**: Transpiled SQL should be reviewed before execution
2. **Test Queries**: Test on sample data when possible
3. **Check Functions**: Verify that all functions are correctly mapped
4. **Handle Edge Cases**: Some Snowflake-specific features may need manual conversion
5. **Use AST Inspection**: For complex transformations, inspect and modify the AST programmatically

---

## Resources

- **SQLGlot Documentation**: https://github.com/tobymao/sqlglot
- **Snowflake SQL Reference**: https://docs.snowflake.com/en/sql-reference/
- **BigQuery SQL Reference**: https://cloud.google.com/bigquery/docs/reference/standard-sql